In [43]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [44]:
df = pd.read_csv('POwer generation.csv')

print(df.dtypes)

df.head()

Day of Year                               int64
Year                                      int64
Month                                     int64
Day                                       int64
First Hour of Period                      int64
Is Daylight                                bool
Distance to Solar Noon                  float64
Average Temperature (Day)                 int64
Average Wind Direction (Day)              int64
Average Wind Speed (Day)                float64
Sky Cover                                 int64
Visibility                              float64
Relative Humidity                         int64
Average Wind Speed (Period)             float64
Average Barometric Pressure (Period)    float64
Power Generated                           int64
dtype: object


,Day of Year,Year,Month,Day,First Hour of Period,Is Daylight,Distance to Solar Noon,Average Temperature (Day),Average Wind Direction (Day),Average Wind Speed (Day),Sky Cover,Visibility,Relative Humidity,Average Wind Speed (Period),Average Barometric Pressure (Period),Power Generated
0,245,2008,9,1,1,False,0.859897,69,28,7.5,0,10.0,75,8.0,29.82,0
1,245,2008,9,1,4,False,0.628535,69,28,7.5,0,10.0,77,5.0,29.85,0
2,245,2008,9,1,7,True,0.397172,69,28,7.5,0,10.0,70,0.0,29.89,5418
3,245,2008,9,1,10,True,0.165810,69,28,7.5,0,10.0,33,0.0,29.91,25477
4,245,2008,9,1,13,True,0.065553,69,28,7.5,0,10.0,21,3.0,29.89,30069


In [45]:
mean_value = df['Average Wind Speed (Period)'].mean()
df['Average Wind Speed (Period)'] = df['Average Wind Speed (Period)'].fillna(mean_value)

In [46]:
df['Is Daylight'] = df['Is Daylight'].map({True:1,False:0})

In [47]:
X = df.iloc[:,1:-1]
y = df.iloc[:,-1]

In [48]:
X

,Year,Month,Day,First Hour of Period,Is Daylight,Distance to Solar Noon,Average Temperature (Day),Average Wind Direction (Day),Average Wind Speed (Day),Sky Cover,Visibility,Relative Humidity,Average Wind Speed (Period),Average Barometric Pressure (Period)
0,2008,9,1,1,0,0.859897,69,28,7.5,0,10.0,75,8.0,29.82
1,2008,9,1,4,0,0.628535,69,28,7.5,0,10.0,77,5.0,29.85
2,2008,9,1,7,1,0.397172,69,28,7.5,0,10.0,70,0.0,29.89
3,2008,9,1,10,1,0.165810,69,28,7.5,0,10.0,33,0.0,29.91
4,2008,9,1,13,1,0.065553,69,28,7.5,0,10.0,21,3.0,29.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,2009,8,31,10,1,0.166453,63,27,13.9,4,10.0,75,10.0,29.93
2916,2009,8,31,13,1,0.064020,63,27,13.9,1,10.0,66,15.0,29.91
2917,2009,8,31,16,1,0.294494,63,27,13.9,2,10.0,68,21.0,29.88
2918,2009,8,31,19,1,0.524968,63,27,13.9,2,10.0,81,17.0,29.87


In [49]:
y

0           0
1           0
2        5418
3       25477
4       30069
        ...  
2915     6995
2916    29490
2917    17257
2918      677
2919        0
Name: Power Generated, Length: 2920, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [51]:
from sklearn.ensemble import AdaBoostRegressor

ada = AdaBoostRegressor()

In [52]:
params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5, 1.0],
    'loss': ['linear','square','exponential']  
}


In [53]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=ada,param_grid=params,verbose=2,n_jobs=-1,cv=5)

In [54]:
grid.fit(X_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


,estimator,AdaBoostRegressor()
,param_grid,"{'learning_rate': [0.01, 0.1, ...], 'loss': ['linear', 'square', ...], 'n_estimators': [50, 100, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,estimator,None


In [57]:
y_pred = grid.predict(X_test)

In [59]:
from sklearn.metrics import r2_score , mean_squared_error , mean_absolute_error

print(r2_score(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))


0.8601868496807902
14507532.942028983
2141.268654552883
